In [1]:
import pandas as pd
import spacy
from tqdm import tqdm
from sklearn.metrics import f1_score
tqdm.pandas()
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [2]:
nlp = spacy.load('ru_core_news_lg')

In [3]:
def create_docbin(data):
    db = DocBin()
    global categories
    for i in range(data.shape[0]):    
        doc = nlp.make_doc(str(data["tokenized_str"][i]))
        doc.cats = {category: 0 for category in categories}
        doc.cats[data["category"][i]] = 1
        db.add(doc)
    return db

In [4]:
def get_prection(text):
    doc = nlp(text)
    scores = doc.cats
    return max(scores, key=scores.get)

In [7]:
len(df1['clear_text'][0])

4419

In [8]:
len(df['clear_text'][0])

5796

In [5]:
df1 = pd.read_csv('../raw_data/data_prep.csv')

In [6]:
df = pd.read_csv('../raw_data/data_prep_last.csv')

In [16]:
df = df.reset_index(drop=True)

In [17]:
kf = KFold(n_splits=5)
fold_generator = kf.split(df)

In [9]:
categories = df['category'].unique()

In [34]:
res = []

for train, test in kf.split(df):
    train = df.iloc[train]
    dev = df.iloc[test]

    train = train.reset_index(drop=True)
    dev = dev.reset_index(drop=True)

    nlp = spacy.load('ru_core_news_lg')

    create_docbin(dev).to_disk("../cat/dev.spacy")
    create_docbin(train).to_disk("../cat/train.spacy")

    ! python -m spacy train ../cat/config.cfg --output ../cat/cat_model --paths.train ../cat/train.spacy --paths.dev ../cat/dev.spacy

    nlp = spacy.load('../cat/cat_model/model-best/')

    dev['y'] = dev['tokenized_str'].apply(get_prection)

    res.append(f1_score(dev['category'], dev['y'], average='weighted'))

    print(f1_score(dev['category'], dev['y'], average='weighted'))

    


ℹ Saving to output directory: ..\cat\cat_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       52.63    0.53
  0     200          24.11       70.08    0.70
  0     400          20.15       68.97    0.69
  0     600          18.44       73.79    0.74
  0     800          16.75       76.76    0.77
  1    1000          16.88       83.68    0.84
  1    1200           8.20       83.07    0.83
  1    1400           7.71       83.05    0.83
  1    1600           7.50       84.03    0.84
  1    1800           6.44       86.25    0.86
  2    2000           5.60       86.46    0.86
  2    2200           2.44       87.67    0.88
  2    2400           3.01       88.65 

In [35]:
res


[0.6829097875062283,
 0.7513352308880721,
 0.6645265306770053,
 0.713156695316463,
 0.5352995523119479]

In [10]:
train, dev = train_test_split(df, test_size=0.2, random_state=42)


In [11]:
train = train.reset_index(drop=True)
dev = dev.reset_index(drop=True)

In [12]:
train.shape, dev.shape

((960, 8), (240, 8))

In [8]:
! python -m spacy init fill-config ../cat/base_config.cfg ../cat/config.cfg

✔ Auto-filled config with all values
✔ Saved config
..\cat\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [13]:
create_docbin(dev).to_disk("../cat/dev.spacy")
create_docbin(train).to_disk("../cat/train.spacy")

! python -m spacy train ../cat/config.cfg --output ../cat/cat_model --paths.train ../cat/train.spacy --paths.dev ../cat/dev.spacy

nlp = spacy.load('../cat/cat_model/model-best/')

dev['y'] = dev['tokenized_str'].apply(get_prection)

print(f1_score(dev['category'], dev['y'], average='weighted'))

ℹ Saving to output directory: ..\cat\cat_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       56.87    0.57
  0     200          24.01       73.37    0.73
  0     400          19.98       75.81    0.76
  0     600          20.61       77.15    0.77
  0     800          15.21       78.03    0.78
  1    1000          15.01       81.00    0.81
  1    1200           6.83       84.29    0.84
  1    1400           6.13       83.38    0.83
  1    1600           8.81       83.18    0.83
  1    1800           8.98       86.34    0.86
  2    2000           5.87       87.84    0.88
  2    2200           3.80       87.48    0.87
  2    2400           2.64       86.38 

In [27]:
! python -m spacy evaluate ../cat/cat_model/model-best/ --output ../cat/metrics.json ../cat/dev.spacy

ℹ Using CPU

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   94.97 
SPEED                 673881


=========================== Textcat F (per label) ===========================

           P       R       F
BB     72.41   91.30   80.77
AAA    96.43   84.38   90.00
AA     85.29   76.32   80.56
A      83.13   83.13   83.13
BBB    94.29   61.11   74.16
B     100.00   57.14   72.73
C     100.00   66.67   80.00


======================== Textcat ROC AUC (per label) ========================

      ROC AUC
BB       0.99
AAA      0.98
AA       0.93
A        0.95
BBB      0.93
B        0.93
C        0.93

✔ Saved results to ..\cat\metrics.json


In [35]:
dev['category'].value_counts()

A      41
BBB    28
AA     20
AAA    13
BB     12
B       4
C       2
Name: category, dtype: int64

In [36]:
train['category'].value_counts()

A      345
BBB    242
AA     179
AAA    140
BB     124
B       40
C       10
Name: category, dtype: int64

In [12]:
categories = df['category'].unique()

In [13]:
categories

array(['BB', 'AAA', 'AA', 'A', 'BBB', 'B', 'C'], dtype=object)

In [41]:
create_docbin(dev).to_disk("../cat/dev.spacy")

In [42]:
create_docbin(train).to_disk("../cat/train.spacy")

In [10]:
nlp = spacy.load('../cat/cat_model/model-best/')

In [11]:
df['tokenized_str'][5]

'подтвердить кредитный рейтинг компания уровень    март рейтинговый агентство подтвердить рейтинг кредитоспособность нефинансовый компания уровень прогноз рейтинг стабильный группа являться вертикально интегрировать металлургический горнодобывающей компания основной актив группа структурно состоять два дивизион включать горнодобывающие актив оценка риск профиль отрасль оказывать умеренно позитивный влияние рейтинг агентство оценивать барьер вход металлургический отрасль высокий отрасль подверженный цикличность сдерживать оценка фактор устойчивость внешний шокам поддерживаться способность удерживать комфортный обслуживание долговой метрики высокий сравнение среднеотраслевой значение маржинальность бизнес низкий точка цикл стальной отрасль вертикально интегрировать модель бизнес высокий уровень самообеспеченность основный сырьё позволять группа первый децили глобальный кривой затрата учёт вертикальный интеграция поддерживать оценка рыночный конкурентный позиция группа являться крупный ро

In [12]:
df.iloc[5]

Unnamed: 0                                                      10
Id                                                              11
pr_txt           «Эксперт РА» подтвердил кредитный рейтинг комп...
category                                                       AAA
rating                                                         AAA
clear_text       подтвердить кредитный рейтинг компания на уров...
tokenized        ['подтвердить', 'кредитный', 'рейтинг', 'компа...
tokenized_str    подтвердить кредитный рейтинг компания уровень...
Name: 5, dtype: object

In [13]:
doc = nlp(df['tokenized_str'][5])

In [14]:
doc.cats

{'BB': 0.00022462634660769254,
 'AAA': 0.9858864545822144,
 'AA+': 8.027573130675592e-06,
 'A': 6.041466986062005e-05,
 'BB+': 2.1728023057221435e-05,
 'A-': 0.00011630508379312232,
 'A+': 0.006935281213372946,
 'BBB': 6.68603115627775e-06,
 'BBB+': 0.0037035911809653044,
 'B-': 2.061153470123145e-09,
 'AA': 3.038844806724228e-05,
 'B+': 9.058681627038823e-08,
 'BB-': 0.0006843357696197927,
 'AA-': 0.00012125987268518656,
 'C': 1.3159824341357762e-08,
 'B': 2.061153470123145e-09,
 'BBB-': 1.037518995872233e-05}

In [15]:
from sklearn.metrics import classification_report

In [16]:
docs = nlp.pipe(dev)

In [ ]:
(scores, key=scores.get)

In [18]:
def get_prection(text):
    doc = nlp(text)
    scores = doc.cats
    return max(scores, key=scores.get)

In [19]:
dev['y'] = dev['tokenized_str'].apply(get_prection)

In [21]:
print(classification_report(dev['rating'], dev['y'], target_names=categories))

              precision    recall  f1-score   support

          BB       0.33      0.28      0.30        18
         AAA       0.50      0.57      0.53        28
         AA+       0.66      0.68      0.67        37
           A       0.83      0.77      0.80        13
         BB+       0.77      0.77      0.77        13
          A-       0.67      0.50      0.57        12
          A+       0.82      0.88      0.85        32
         BBB       0.00      0.00      0.00         2
        BBB+       0.67      0.50      0.57         4
          B-       0.00      0.00      0.00         1
          AA       0.33      0.44      0.38         9
          B+       0.62      0.56      0.59         9
         BB-       0.20      0.20      0.20         5
         AA-       0.62      0.42      0.50        19
           C       0.54      0.57      0.55        23
           B       0.35      0.58      0.44        12
        BBB-       1.00      0.67      0.80         3

    accuracy              

c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\wilte\OneDrive\Рабочий стол\projects\rating-agency-analyser\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

In [20]:
categories 

,Unnamed: 0,Id,pr_txt,category,rating,clear_text,tokenized,tokenized_str,y
0,650,652,"НКР присвоило ""СР-ТЭК"" кредитный рейтинг <rati...",B,B+,присвоить кредитный рейтинг со стабильный прог...,"['присвоить', 'кредитный', 'рейтинг', 'стабиль...",присвоить кредитный рейтинг стабильный прогноз...,A-
1,549,550,Кредитный рейтинг АО «ИНК-Капитал» (далее — ...,A,A+,кредитный рейтинг далее компания обусловить си...,"['кредитный', 'рейтинг', 'компания', 'обуслови...",кредитный рейтинг компания обусловить сильный ...,AAA
2,227,228,«Эксперт РА» присвоил кредитный рейтинг АО «ЮГ...,AA,AA-,эксперт ра присвоить кредитный рейтинг на уров...,"['эксперт', 'ра', 'присвоить', 'кредитный', 'р...",эксперт ра присвоить кредитный рейтинг уровень...,AA
3,875,883,«Эксперт РА» присвоил кредитный рейтинг компан...,BB,BB-,эксперт ра присвоить кредитный рейтинг компани...,"['эксперт', 'ра', 'присвоить', 'кредитный', 'р...",эксперт ра присвоить кредитный рейтинг компани...,BBB-
4,124,125,«Эксперт РА» подтвердил кредитный рейтинг ПАО ...,A,A+,подтвердить кредитный рейтинг на уровень ноябр...,"['подтвердить', 'кредитный', 'рейтинг', 'урове...",подтвердить кредитный рейтинг уровень ноябрь р...,A+
...,...,...,...,...,...,...,...,...,...
235,766,770,«Эксперт РА» пересмотрел рейтинг компании «Дет...,AA,AA-,пересмотреть рейтинг компания по обновить мето...,"['пересмотреть', 'рейтинг', 'компания', 'обнов...",пересмотреть рейтинг компания обновить методол...,AA-
236,555,556,Повышение кредитного рейтинга АО «ГК «Пионер»...,A,A-,повышение кредитный рейтинг далее компания обу...,"['повышение', 'кредитный', 'рейтинг', 'компани...",повышение кредитный рейтинг компания обусловит...,BBB+
237,1041,1053,«Эксперт РА» подтвердил рейтинг компании АРКТИ...,AA,AA+,подтвердить рейтинг компания на уровень декабр...,"['подтвердить', 'рейтинг', 'компания', 'уровен...",подтвердить рейтинг компания уровень декабрь р...,A+
238,844,850,Н К Р | нефинансовые компании | золотодобыча Н...,A,A+,н к р нефинансовый компания золотодобыча присв...,"['нефинансовый', 'компания', 'золотодобыча', '...",нефинансовый компания золотодобыча присвоить к...,A+
